In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import requests

In [4]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
movies=pd.read_csv(r"D:\Data Science\movies dataset\tmdb_5000_movies.csv")
credits=pd.read_csv(r"D:\Data Science\movies dataset\tmdb_5000_credits.csv")

In [6]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [7]:
movies=movies.merge(credits,on='title')

In [8]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [10]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [11]:
movies['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

#### Our K

In [12]:
def convert(text):
    list=[]
    value=json.loads(text)
    for i in range(len(value)):
        name=value[i]['name']
        list.append(name)
    return list

In [13]:
movies['keywords']=movies['keywords'].apply(convert)

In [14]:
movies['genres'] = movies['genres'].apply(convert)

In [15]:
movies['cast']=movies['cast'].apply(convert)

In [16]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [17]:
def fetch_director(text):
    list=[]
    value=json.loads(text)
    for i in range(len(value)):
        if value[i]['job']=='Director':
            list.append(value[i]['name'])
    return list

In [18]:
movies['crew']=movies['crew'].apply(fetch_director)

In [19]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [20]:
movies['overview'].isnull().sum()

3

In [21]:
movies.dropna(inplace=True)

In [22]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def stopword_remove(text):
    regex=re.compile('[%s]'% re.escape(string.punctuation))
    text= regex.sub('',text)
    word_tokens= word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return filtered_sentence



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoges\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [24]:
movies['overview']=movies['overview'].apply(stopword_remove)

In [25]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [26]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [27]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[22nd, century, paraplegic, Marine, dispatched...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa, long, believed, dead, come...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[cryptic, message, Bond, ’, past, sends, trail...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, death, District, Attorney, Harvey,...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, warweary, former, military, cap...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [28]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [29]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [30]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

,movie_id,title,tags
0,19995,Avatar,22nd century paraplegic Marine dispatched moon...
1,285,Pirates of the Caribbean: At World's End,Captain Barbossa long believed dead come back ...
2,206647,Spectre,cryptic message Bond ’ past sends trail uncove...
3,49026,The Dark Knight Rises,Following death District Attorney Harvey Dent ...
4,49529,John Carter,John Carter warweary former military captain w...


In [31]:
new['tags'][0]

'22nd century paraplegic Marine dispatched moon Pandora unique mission becomes torn following orders protecting alien civilization Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [32]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [33]:

vector = cv.fit_transform(new['tags']).toarray()

In [34]:
vector.shape

(4806, 5000)

In [35]:
similarity = cosine_similarity(vector)

In [36]:
similarity

array([[1.        , 0.08980265, 0.05892557, ..., 0.02541643, 0.02993422,
        0.        ],
       [0.08980265, 1.        , 0.06350006, ..., 0.02738955, 0.        ,
        0.        ],
       [0.05892557, 0.06350006, 1.        , ..., 0.02695819, 0.        ,
        0.        ],
       ...,
       [0.02541643, 0.02738955, 0.02695819, ..., 1.        , 0.0547791 ,
        0.02353104],
       [0.02993422, 0.        , 0.        , ..., 0.0547791 , 1.        ,
        0.0277137 ],
       [0.        , 0.        , 0.        , ..., 0.02353104, 0.0277137 ,
        1.        ]])

In [37]:
new[new['title'] == 'Batman Begins'].index[0]

119

In [38]:
distance=sorted(list(enumerate(similarity[119])),reverse=True, key= lambda X: X[1])
distance[0:5]


NameError: name 'diatance' is not defined

In [ ]:
def recommend(movie):
    index=new[new['title'] == movie].index[0]
    distances=sorted(list(enumerate(similarity[index])),reverse=True, key= lambda X: X[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)
    
    


In [ ]:
recommend('The Matrix')

The Matrix Revolutions
The Matrix Reloaded
Jupiter Ascending
Sleep Dealer
Æon Flux


In [ ]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [41]:
def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}?api_key=63b6b9cc24aab69e989db4c0c6967eae&language=en-US".format(movie_id)
    data = requests.get(url)
    data = data.json()
    poster_path = data['poster_path']
    full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
    return full_path


In [42]:
def recommend(movie):
    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_movie_names = []
    recommended_movie_posters = []
    for i in distances[1:6]:
        # fetch the movie poster
        movie_id = movies.iloc[i[0]].movie_id
        recommended_movie_posters.append(fetch_poster(movie_id))
        recommended_movie_names.append(movies.iloc[i[0]].title)

    return recommended_movie_names,recommended_movie_posters


In [43]:
recommend('Avatar')

(['Titan A.E.',
  'Small Soldiers',
  'Independence Day',
  "Ender's Game",
  'Predators'],
 ['https://image.tmdb.org/t/p/w500//el2iHk3LTJWfEnwrvcRkvWY501G.jpg',
  'https://image.tmdb.org/t/p/w500//2nuUjSzHsoYlRvTPmLo7m7gCQry.jpg',
  'https://image.tmdb.org/t/p/w500//p0BPQGSPoSa8Ml0DAf2mB2kCU0R.jpg',
  'https://image.tmdb.org/t/p/w500//vrEpLNkv30qw7JiVyorgR6NOWDm.jpg',
  'https://image.tmdb.org/t/p/w500//54LZIRznS0Zjn7ZDXrsGVe3HXei.jpg'])